In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkContext
import json

In [3]:
sc = SparkContext('local[8]', 'task2')

In [4]:
reviews_txt = sc.textFile('asnlib/publicdata/review.json')


In [5]:
business_txt = sc.textFile('asnlib/publicdata/business.json')

In [6]:
reviews_json = reviews_txt.map(lambda x: json.loads(x))

In [ ]:
reviews_json.first()

In [7]:
business_json = business_txt.map(lambda x: json.loads(x))

In [ ]:
business_json.take(1)

In [ ]:
def get_category_business_id(row):
    category_business_list = []
    for category in row['categories'].split(','):
        category_business_list.append((category.strip(), row['business_id']))
    return category_business_list

In [ ]:
# categories = business_json.map(lambda x: ([cat.strip()for cat in x['categories'].split(',')], x['business_id']))


In [ ]:
categories = business_json.filter(lambda x: x.get('categories',  None) is not None).flatMap(get_category_business_id).groupByKey().map(lambda x: (x[0], list(x[1])))

In [ ]:
categories.first()

In [ ]:
reduced_cat = categories.reduceByKey(lambda a,b: a + b)

In [ ]:
reduced_cat.collect()

In [ ]:
business_ratings = reviews_json.map(lambda x: (x['business_id'], x['stars']))

In [ ]:
business_ratings.count()

In [ ]:
grouped_b = business_ratings.groupByKey().map(lambda x: (x[0], list(x[1])))

In [ ]:
grouped_b.take(10)

In [ ]:
joined = business_ratings.join(categories)

In [ ]:
joined.first()

In [15]:
categories = business_json.filter(lambda x: x.get('categories',  None) is not None).flatMap(get_category_business_id)

In [16]:
categories.take(10)

[('1SWheh84yJXfytovILXOAQ', 'Golf'),
 ('1SWheh84yJXfytovILXOAQ', 'Active Life'),
 ('QXAEGFB4oINsVuTFxEYKFQ', 'Specialty Food'),
 ('QXAEGFB4oINsVuTFxEYKFQ', 'Restaurants'),
 ('QXAEGFB4oINsVuTFxEYKFQ', 'Dim Sum'),
 ('QXAEGFB4oINsVuTFxEYKFQ', 'Imported Food'),
 ('QXAEGFB4oINsVuTFxEYKFQ', 'Food'),
 ('QXAEGFB4oINsVuTFxEYKFQ', 'Chinese'),
 ('QXAEGFB4oINsVuTFxEYKFQ', 'Ethnic Food'),
 ('QXAEGFB4oINsVuTFxEYKFQ', 'Seafood')]

In [17]:
joined_rdd = categories.join(review_stars)

In [18]:
joined_rdd.take(10)

[('jZ23B--fu21is2zrWiy4Kg', ('Pet Groomers', 5.0)),
 ('jZ23B--fu21is2zrWiy4Kg', ('Pet Groomers', 5.0)),
 ('jZ23B--fu21is2zrWiy4Kg', ('Pets', 5.0)),
 ('jZ23B--fu21is2zrWiy4Kg', ('Pets', 5.0)),
 ('jZ23B--fu21is2zrWiy4Kg', ('Pet Sitting', 5.0)),
 ('jZ23B--fu21is2zrWiy4Kg', ('Pet Sitting', 5.0)),
 ('jZ23B--fu21is2zrWiy4Kg', ('Pet Services', 5.0)),
 ('jZ23B--fu21is2zrWiy4Kg', ('Pet Services', 5.0)),
 ('2ffee6OI50skuFyEVmOBZQ', ('Eyelash Service', 5.0)),
 ('2ffee6OI50skuFyEVmOBZQ', ('Beauty & Spas', 5.0))]

In [22]:
category_ratings_map = joined_rdd.map(lambda x: x[1]).groupByKey().map(lambda x: (x[0], list(x[1])))

In [26]:
average_ratings = category_ratings_map.mapValues(lambda x: sum(x)/len(x)).sortBy(lambda x: (-x[1], x[0]))

In [28]:
average_ratings.take(50)

[('Astrologers', 5.0),
 ('Audio/Visual Equipment Rental', 5.0),
 ('Bocce Ball', 5.0),
 ('Calabrian', 5.0),
 ('Chinese Martial Arts', 5.0),
 ('Christmas Markets', 5.0),
 ('Crane Services', 5.0),
 ('DIY Auto Shop', 5.0),
 ('Duplication Services', 5.0),
 ('Fireworks', 5.0),
 ('Geneticists', 5.0),
 ('Habilitative Services', 5.0),
 ('Hearing Aids', 5.0),
 ('IP & Internet Law', 5.0),
 ('Indoor Landscaping', 5.0),
 ('Japanese Sweets', 5.0),
 ('Makerspaces', 5.0),
 ('Mobile Home Repair', 5.0),
 ('Outdoor Movies', 5.0),
 ('Pathologists', 5.0),
 ('Public Adjusters', 5.0),
 ('Registry Office', 5.0),
 ('Rodeo', 5.0),
 ('Safety Equipment', 5.0),
 ('Sledding', 5.0),
 ('Snorkeling', 5.0),
 ('Storefront Clinics', 5.0),
 ('Structural Engineers', 5.0),
 ('Surf Schools', 5.0),
 ('Trivia Hosts', 5.0),
 ('Vocal Coach', 5.0),
 ('Water Suppliers', 5.0),
 ('Caricatures', 4.933333333333334),
 ('Taxidermy', 4.933333333333334),
 ('Party Characters', 4.873015873015873),
 ('Boudoir Photography', 4.857142857142857)